# Getting Started with Pilot-Streaming and Edge on TACC

In the first step we need to import all required packages and modules into the Python Path

Pilot-Streaming can be used to manage the Dask and Kafka environments both in the cloud and on the edge. 


`resource`: URL of the Local Resource Manager. Examples:

* `slurm://localhost`: Submit to local SLURM resource manager, e.g. on master node of Wrangler or Stampede
* `slurm+ssh://login1.wrangler.tacc.utexas.edu`: Submit to Wrangler master node SLURM via SSH (e.g. on node running a job)

`type:` The `type` attributes specifies the cluster environment. It can be: `Spark`, `Dask` or `Kafka`.


Depending on the resource there might be other configurations necessary, e.g. to ensure that the correct subnet is used the Spark driver can be configured using various environment variables:   os.environ["SPARK_LOCAL_IP"]='129.114.58.2'

In [1]:
# Pilot-Streaming
import os, sys
import pilot.streaming
sys.modules['pilot.streaming']

RESOURCE_URL_HPC="slurm+ssh://login4.stampede2.tacc.utexas.edu"
WORKING_DIRECTORY=os.path.join(os.environ["HOME"], "work")

RESOURCE_URL_EDGE="ssh://js-17-136.jetstream-cloud.org"
RESOURCE_URL_EDGE="ssh://localhost"
WORKING_DIRECTORY_EDGE="/home/aluckow"

INFO:pykafka.topic:Successfully loaded pykafka.rdkafka extension.


# 1. Dask on Jetstream (pre-launched VM)

In [2]:
import distributed

pilot_compute_description = {
    "resource":RESOURCE_URL_EDGE,
    "working_directory": WORKING_DIRECTORY_EDGE,
    "number_of_nodes": 1,
    "cores_per_node": 1,
    "dask_cores" : 2,
    "project": "TG-MCB090174",
    "queue": "normal",
    "walltime": 359,
    "type":"dask"
}

In [3]:
%%time
dask_pilot = pilot.streaming.PilotComputeService.create_pilot(pilot_compute_description)
dask_pilot.wait()

DEBUG:root:Run python Args: ['-m', 'pilot.plugins.dask.bootstrap_dask', ' -p ', '2']
DEBUG:root:URL: ssh://localhost Host: localhost
DEBUG:root:Execute: ['ssh', 'localhost', '/bin/date']
DEBUG:root:Test Job succeeded


Working Directory: /home/aluckow
dask-1fbc94d0-9dd5-11ea-96ae-55fd637a5036/home/aluckow
Working Directory: /home/aluckow/dask-1fbc94d0-9dd5-11ea-96ae-55fd637a5036
Start Dask Cluster: dask-ssh --nthreads 1 --remote-dask-worker distributed.cli.dask_worker localhost


DEBUG:root:Job State : running
DEBUG:root:Results of scheduler startup file check: True
DEBUG:root:**** Job: pilot-streaming-ssh1fbd4718-9dd5-11ea-96ae-55fd637a5036 State: running
DEBUG:root:looking for Dask startup state at: /home/aluckow/dask-1fbc94d0-9dd5-11ea-96ae-55fd637a5036
DEBUG:root:Results of scheduler startup file check: True
DEBUG:root:Results of scheduler startup file check: True
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector


**** Job: pilot-streaming-ssh1fbd4718-9dd5-11ea-96ae-55fd637a5036 State: running
init distributed client
Connect to Dask: tcp://localhost:8786
{'type': 'Scheduler', 'id': 'Scheduler-609771f3-c949-420f-a8d0-aca4f1b67638', 'address': 'tcp://172.27.152.6:8786', 'services': {'dashboard': 8787}, 'workers': {'tcp://127.0.0.1:38405': {'type': 'Worker', 'id': 'tcp://127.0.0.1:38405', 'host': '127.0.0.1', 'resources': {}, 'local_directory': '/home/aluckow/dask-worker-space/worker-jc5zch54', 'name': 'tcp://127.0.0.1:38405', 'nthreads': 1, 'memory_limit': 2068051968, 'last_seen': 1590334937.3685338, 'services': {'dashboard': 43755}, 'metrics': {'cpu': 2.0, 'memory': 104726528, 'time': 1590334937.3671763, 'read_bytes': 1009.6416922428838, 'write_bytes': 1009.6416922428838, 'num_fds': 23, 'executing': 0, 'in_memory': 0, 'ready': 0, 'in_flight': 0, 'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}}, 'nanny': 'tcp://127.0.0.1:32927'}}}
CPU times: user 34.2 ms, sys: 19.7 ms, total: 53.9 ms

In [4]:
dask_pilot.get_details()

DEBUG:root:Results of scheduler startup file check: True


{'master_url': 'tcp://localhost:8786', 'web_ui_url': 'http://localhost:8787'}

In [5]:
import distributed
dask_client  = distributed.Client(dask_pilot.get_details()['master_url'])
dask_client.scheduler_info()

DEBUG:root:Results of scheduler startup file check: True
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:asyncio:Using selector: EpollSelector


{'type': 'Scheduler',
 'id': 'Scheduler-609771f3-c949-420f-a8d0-aca4f1b67638',
 'address': 'tcp://172.27.152.6:8786',
 'services': {'dashboard': 8787},
 'workers': {'tcp://127.0.0.1:38405': {'type': 'Worker',
   'id': 'tcp://127.0.0.1:38405',
   'host': '127.0.0.1',
   'resources': {},
   'local_directory': '/home/aluckow/dask-worker-space/worker-jc5zch54',
   'name': 'tcp://127.0.0.1:38405',
   'nthreads': 1,
   'memory_limit': 2068051968,
   'last_seen': 1590334943.3685687,
   'services': {'dashboard': 43755},
   'metrics': {'cpu': 2.0,
    'memory': 104984576,
    'time': 1590334943.3671722,
    'read_bytes': 1155.4919950392598,
    'write_bytes': 1211.4673858024073,
    'num_fds': 23,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://127.0.0.1:32927'}}}

In [6]:
dask_client.gather(dask_client.map(lambda a: a*a, range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

# 2. Start Kafka Cluster

# 3. Start Stream Processing on Kafka/Dask